In [1]:
import os
# os.system('cls')  # On Windows System

from IPython import get_ipython
get_ipython().run_line_magic('reset','-sf')

In [33]:
import numpy as np # Import numpy
import matplotlib.pyplot as plt # Import pyplot
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
# plt.close('all')

In [3]:
# import the features and the labels for the training data and the test data
# import the training data
# train_data = pd.read_csv('train.csv')
# test_data = pd.read_csv('test.csv')

train_data = np.loadtxt('train.csv', delimiter=',', skiprows=1) 
test_data = np.loadtxt('test.csv', delimiter=',', skiprows=1)

train_data = train_data[:800]
test_data = test_data[:200]

In [64]:
train_data[100]

array([1.15521633e+03, 1.56950516e+03, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.09825450e+02,
       0.00000000e+00, 1.00000000e+00, 4.25000000e+00])

In [7]:
train_data.shape

(800, 11)

In [8]:
test_data.shape

(200, 11)

In [10]:
split_index = int(0.5 * len(test_data))


In [11]:
split_index

100

In [13]:
eval_data = test_data[:split_index]
test_data = test_data[split_index:]

In [14]:
eval_data.shape

(100, 11)

In [16]:
test_data.shape

(100, 11)

In [17]:
train_data.shape

(800, 11)

In [18]:
# normalize the training data using z-normalization 
mean = np.mean(train_data, axis=0)
std = np.std(train_data, axis=0)
normalized_train_data = (train_data - mean)/std

In [19]:
# normalize the training data using z-normalization but using the training data mean and standard deviation
# normalize the data using z-normalization 
normalized_test_data = (test_data - mean)/std

In [20]:
# # separate the training features from the labels 
# xtrain = train_data.iloc[:, :10]
# ytrain = train_data.iloc[:,-1:]

In [21]:
# separeate the test features from the labels
# xtest= test_data.iloc[:, :10]
# ytest = test_data.iloc[:,-1:]

In [ ]:
# sigmoid() function (activation function)
def sigmoid(x):
    fAct = 1/(1+np.exp(-1*x))
    return fAct


In [73]:
def leakyReLu(x, alpha=0.01):
    fAct = np.where(x>0,x, alpha*x)
    return fAct

In [74]:
def derived_leakyReLu(x, alpha=0.01):
    fAct = np.where(x > 0, 1, alpha)
    return fAct

In [75]:
# Derivation of the activation fucntion
def derived_sigmoid(x):
    fAct = 1/(1+np.exp(-1*x))
    DfAct = fAct*(1-fAct)
    return DfAct

In [76]:
# Generate m*n matrix, initial values are zeros
def makematrix(m, n, fill=0.0):
    a = []
    for i in range(m):
        a.append([fill]*n)
    return np.array(a)

In [77]:
# Random data
def random_number(a,b):
    return (b-a)*np.random.normal()+a

In [78]:
normalized_train_data.shape

(800, 11)

In [79]:
len(normalized_test_data[:,0])

100

In [83]:
#BP Neural Networks
class BPNN:
    def __init__(self, num_in, num_hidden, num_out):
        # Nodes of input, hidden, output layers
        self.num_in = num_in + 1  # Add one bias node
        self.num_hidden = num_hidden + 1   # Add one bias node
        self.num_out = num_out
        
        # Activate all nodes (vector)
        self.active_in = np.array([1.0]*self.num_in) #creating a numpy array with the values of 1.0
        self.active_hidden = np.array([1.0]*self.num_hidden)
        self.active_out = np.array([1.0]*self.num_out)
        
        # Create weight matrices
        self.wight_in = makematrix(self.num_in, self.num_hidden-1) 
        self.wight_out = makematrix(self.num_hidden, self.num_out)
        
        # Initialize weights
        for i in range(self.num_in):
            for j in range(self.num_hidden-1):
                self.wight_in[i][j] = random_number(0.1, 0.1)
        for i in range(self.num_hidden):
            for j in range(self.num_out):
                self.wight_out[i][j] = random_number(0.1, 0.1)

        # Initialize bias
        for j in range(self.num_hidden-1):
            self.wight_in[0][j] = 0.1 
        for j in range(self.num_out):
            self.wight_out[0][j] = 0.1
              
    # Feed-forward
    def Feedforward(self, inputs):
        if np.shape(inputs)[1] != self.num_in-1:
            raise ValueError('Incorrect input numbers')
        
        # Input layer values
        self.active_in[1:self.num_in]=inputs #here we only get the last x withon the first which is the bias
        
        # Hidden layer values
        self.sum_hidden=np.dot(self.wight_in.T,np.array([self.active_in]).T)
        self.active_hidden = np.vstack( (1, sigmoid(self.sum_hidden)) )   # Activation function
        
        # Output layer values
        self.sum_out=np.dot(self.wight_out.T,self.active_hidden)
        self.active_out = self.sum_out # or sigmoid(self.sum_out)
        return self.active_out
 
    # Backpropagation
    def errorbackpropagate(self, targets):   # lr is the learning rate
        if self.num_out==1:
            targets=targets
        if np.shape(targets)[1] != self.num_out:
            raise ValueError('Incorrect Output numbers')
        # Cost function
        self.error=(1/2)*np.dot((self.active_out-targets.T).T,(self.active_out-targets.T))
        
        # dJ/dw_out
        self.gradient_out = (self.active_out-targets.T) #*derived_sigmoid(self.sum_out)#dJ/dx
        self.gradient_w_out = np.dot(self.gradient_out,self.active_hidden.T).T #dx/dw_out
        
        # dJ/dw_in
        self.gradient_hidden = np.dot(self.wight_out[1:],self.gradient_out)*derived_sigmoid(self.sum_hidden) #dJ/dx
        self.gradient_w_in = np.dot(self.gradient_hidden,np.array([self.active_in])).T #dx/dw_in
       
        return self.error

    def train(self, pattern, itera=1000, lr = 0.5):
        # error = 100000
        i=0
        # while(error 10):
        for i in range(itera):
            Gradient_out = 0.0
            Gradient_in = 0.0
            error = 0
            i+=1
            for j in pattern:
                inputs = np.array([j[0:self.num_in-1]])
                
                targets = np.array([j[self.num_in-1:]])
                self.Feedforward(inputs)
                self.errorbackpropagate(targets)
                Gradient_out = Gradient_out + self.gradient_w_out
                Gradient_in = Gradient_in + self.gradient_w_in
            
                error = error + self.error
                
            [Tm,Tn] = np.shape(pattern)
            self.wight_out = self.wight_out - lr*Gradient_out/Tm
            self.wight_in = self.wight_in - lr*Gradient_in/Tm 
            
            if i % 100 == 0:
                print('INFO|Error: %-.5f|Iteration: %d' %(error,i))
            if i % 1000 == 0:
                # Get current date in YYYYMMDD format
                current_date = datetime.now().strftime('%Y%m%d%hh%mm') 

                # Create the files to store the weights with date and iteration number
                weights_in_file = f'weights/weights_in_{current_date}_{i}.npy'
                weights_out_file = f'weights/weights_out_{current_date}_{i}.npy'
                np.save(weights_in_file, self.wight_in)
                np.save(weights_out_file, self.wight_in)
        
    # Val prediction
    def predict(self, patterns):
        Pre = []
        for i in patterns:
            inputs = np.array([i[0:self.num_in-1]])
            Pre.append( self.Feedforward(inputs) )
            # print(Pre)
        return Pre   


n = BPNN(10, 16, 1) #Create neural networK 
n.train(normalized_train_data) # Training BP

#After training predict using the training set
y_t = n.predict(normalized_train_data)
y_true_denorm = y_t * std + mean
y_t_true = train_data[:,-1]
print(y_t_true[0])
epsilon = 1e-6  # Small value to avoid division by zero
y_true_safe = np.where(y_t_true == 0, epsilon, y_t_true)

y_t_pred = np.zeros((len(normalized_train_data[:,0]),))

for i in range(len(y_true_denorm)):
  y_t_pred[i] = y_true_denorm[i][0,0]

rmse = np.sqrt(np.mean((y_t_true - y_t_pred) ** 2))
mae = np.mean(np.abs(y_t_true - y_t_pred)) 
mre = np.mean(np.abs((y_t_true - y_t_pred) / y_true_safe))
print(f'rmse: {rmse}')
print(f'mae: {mae}')
print(f'mre: {mre}')

# n.weights() # check weights


INFO|Error: 191.16854|Iteration: 100
INFO|Error: 186.78931|Iteration: 200
INFO|Error: 184.55507|Iteration: 300
INFO|Error: 183.21269|Iteration: 400
INFO|Error: 182.29638|Iteration: 500
INFO|Error: 181.59965|Iteration: 600
INFO|Error: 181.04293|Iteration: 700
INFO|Error: 180.58492|Iteration: 800
INFO|Error: 180.19937|Iteration: 900
INFO|Error: 179.86878|Iteration: 1000
11.45
rmse: 1583.1798620474917
mae: 1492.6017815878292
mre: 1057313.8523133888


In [ ]:

plt.scatter(y_t_true, y_t_pred)
plt.plot([min(y_t_true), max(y_t_true)], [min(y_t_true), max(y_t_true)], color='red', linestyle='--')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.title('Model Predictions vs True Values')
plt.show()

In [ ]:
187.65572
179.86878